In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np


模型方案：采用bootstrap进行重采样，利用标准逻辑斯蒂模型进行拟合


模型结果：准确率77%。假阳性率35%，假阴性率16%，精确率83%，召回率83%（比率定义见后文）

因子重要性排序：Current cycle sales,  Number of associated videos,  Unit price,  Related influencers,  Related live streaming,     Number of historical products(系数为负),  Monthly estimated revenue (million)(系数为负)。 排名越靠前意味着对在AMZ上架商品的概率的影响权重越大。


具体而言，可以有如下两种理解方式：第一、current cycle sale增加一个单位对概率的影响比在Unit price增加一个单位所施加的影响更大；第二，A商家比B商家 current cycle sales高出x，比B商家Unit price低x，其他条件一致的情况下，A商家更有可能上架AMZ

模型评价：总体准确率较高，假阳性率较高主要系TTS sheet1数据表的阴性阳性结构不平衡所致（阴性占比仅三分之一），导致模型倾向于采用宁可错杀，一个不漏的策略

模型结论：业务资源充足，追求快速增长的条件下本模型适用，着重将资源利用在高Current cycle sales,  Number of associated videos,  Unit price,  Related influencers,  Related live streaming的商家上，能够更加高效的利用资源吸引商户入住。

读取数据

In [13]:
ori_dataset=pd.read_excel("TTS selection.xlsx")
ori_dataset.head()

,TTS Brand,Product category,Monthly estimated revenue (million),Current cycle sales,Number of historical products,Number of associated videos,Related influencers,Related live streaming,Unit price,Amazon presence (Yes or no),AMZ presence
0,Tarte Cosmetics,Beauty&Personal Care,7.95,291.27,283,15.31,4.45,5.81,27.29,Y,1
1,medicube US Store,Beauty&Personal Care,4.24,133.74,150,83.08,8.65,6.81,31.76,Y,1
2,wavytalk,Beauty&Personal Care,3.02,86.73,71,20.32,2.95,2.37,34.87,Y,1
3,Dr.Melaxin Global,Beauty&Personal Care,2.47,2.47,69,39.06,5.14,3.74,40.13,Y,1
4,Anua Store US,Beauty&Personal Care,2.42,51.75,87,26.60,2.25,3.74,46.69,Y,1


关键数据预处理

In [14]:
#定义目标变量
target=ori_dataset["AMZ presence"]
#设置有效特征：
column=ori_dataset.columns.to_list()
feature=ori_dataset[[name for name in column[2:9]]]

数据量不够，需要重采样，用最基本的bootstrap

In [15]:
#设置参数
n=100#重采样次数
samplesize=len(ori_dataset)#每次采样的规模，进行有放回的采样。
#这个samplesize意味着，我每次从原始数据中抽一个点出来记录进临时sample中，然后放回去再抽，直到sample被填满
#更新的dataframe，这里是我们进行重采样后的，用来训练模型的样本集
bootstrap_data=pd.DataFrame(columns=feature.columns)
bootstrap_target=pd.Series(dtype=int)
for i in range(n):
    #有放回随机采样
    slice=np.random.choice(range(samplesize),size=samplesize,replace=True)
    #填进我们的重采样后的样本集
    sampledata=feature.iloc[slice].copy()
    sampletarget=target.iloc[slice].copy()
    sampledata["bootstrap_round"]=i#重采样轮次标识
    bootstrap_data=pd.concat([bootstrap_data,sampledata],ignore_index=True)
    bootstrap_target=pd.concat([bootstrap_target,sampletarget],ignore_index=True)
#重采样结束，合并生成新的数据集
bootstrap_data["AMZ_presence"]=bootstrap_target
print(bootstrap_data)

      Monthly estimated revenue (million)  Current cycle sales  \
0                                 3.02000                86.73   
1                                 1.22000                98.27   
2                                 2.42000                51.75   
3                                 1.79000                76.39   
4                                 1.93000                35.52   
...                                   ...                  ...   
2995                              1.79000                76.39   
2996                              1.40000                73.17   
2997                              1.45000                31.98   
2998                              0.95492                21.68   
2999                              1.04000                23.60   

     Number of historical products  Number of associated videos  \
0                               71                        20.32   
1                               52                        20.09   
2     

C:\Users\16333\AppData\Local\Temp\ipykernel_14248\3168027661.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  bootstrap_data=pd.concat([bootstrap_data,sampledata],ignore_index=True)


数据集划分和拟合模型前准备

In [16]:
#设立X，Y
AMZ_presence=bootstrap_data["AMZ_presence"]
column=bootstrap_data.columns
X=bootstrap_data[[name for name in column[0:7]]]
#测试集和训练集
X_train,X_test,AMZ_presence_train,AMZ_presence_test=train_test_split(X,AMZ_presence,
                                                                     test_size=0.2,random_state=42,stratify=AMZ_presence)
#进行标准化处理，Z-score方法（就是x-mean/standard deviation）
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.fit_transform(X_test)

拟合模型，训练和模型效力检查

In [17]:
#创建模型
model=LogisticRegression(
    penalty="l2",#正则化防止过拟合——即过度训练
    C=1.0,#正则化强度，越小值越高，防止欠拟合问题
    solver="lbfgs",#逻辑斯蒂常用优化算法
    max_iter=1000#迭代次数
    )
#拟合模型
model.fit(X_train_scaled,AMZ_presence_train)
#进行模型预测（测试）
AMZ_presence_pred=model.predict(X_test_scaled)
AMZ_presence_prob=model.predict_proba(X_test_scaled)[:,1]
#展示准确率
print(f"准确率: {accuracy_score(AMZ_presence_pred, AMZ_presence_test):.2f}")
#展示混淆矩阵——判断模型性能
print("混淆矩阵:")
print(confusion_matrix(AMZ_presence_test,AMZ_presence_pred))
#混淆矩阵怎么看？
#以这里的2×2矩阵为例。
#第一行第一列的元素是实际上是0，模型判断也为0的个数(TN)——真阴性
#第一行第二列的元素是实际上是0，但模型判断为1的个数(FP)——假阳性
#第二行第一列的元素是实际上是1，但模型判断为0的个数(FN)——假阴性
#第二行第二列的元素是实际上是1，模型判断也为1的个数(TP)——真阳性
#在这个例子里，该模型判断的准确率较高。

准确率: 0.85
混淆矩阵:
[[159  30]
 [ 61 350]]


In [18]:
#关键指标计算
matrix=confusion_matrix(AMZ_presence_test,AMZ_presence_pred)
TN=matrix[0][0]
FP=matrix[0][1]
FN=matrix[1][0]
TP=matrix[1][1]
FPR=FP/(FP+TN)#假阳性率，错误的把不上架的归类进上架
FNR=FN/(FN+TP)#假阴性率，错误的把要上架归类为不上架
precision=TP/(TP+FP)#精确率，预测上架中实际上真的上架的比率
recall=TP/(TP+FN)#召回率，实际真的上架中，预测也上架的比率
print("假阳性率:",FPR)
print("假阴性率:",FNR)
print("精确率:",precision)
print("召回率:",recall)

假阳性率: 0.15873015873015872
假阴性率: 0.14841849148418493
精确率: 0.9210526315789473
召回率: 0.851581508515815


从这里可以看到，模型最大的问题是假阳性太高，通过检查数据情况可以发现，主要问题在于数据的结构性不平衡，负样本比例过少，导致模型倾向于采用“宁可错杀也不放过的方案”

各因子重要程度

In [19]:
importance = pd.DataFrame({
    'Feature': feature.columns,
    'Coefficient': model.coef_[0]
}).sort_values('Coefficient', ascending=False)

print("\n特征重要性:")
print(importance)


特征重要性:
                               Feature  Coefficient
1                  Current cycle sales     4.816989
3          Number of associated videos     3.207504
6                           Unit price     3.148013
4                  Related influencers     1.645597
5               Related live streaming     0.979193
2        Number of historical products    -0.273402
0  Monthly estimated revenue (million)    -3.385936
